In [30]:
import pandas as pd
from pycaret.regression import *

In [31]:
train = pd.read_csv("train_meanprice.csv")
test= pd.read_csv("test_meanprice.csv")

In [32]:
exp = setup(train, target="n", normalize=False, remove_outliers=False, categorical_features=["Month", "shopID", "itemID"], train_size=0.7, profile=True)

,Description,Value
0,Session id,7997
1,Target,n
2,Target type,Regression
3,Original data shape,"(67320, 5)"
4,Transformed data shape,"(67320, 33)"
5,Transformed train set shape,"(47124, 33)"
6,Transformed test set shape,"(20196, 33)"
7,Numeric features,1
8,Categorical features,3
9,Preprocess,True


Loading profile... Please Wait!
Profiler Failed. No output to show, continue with modeling.


In [33]:
lgbm = create_model("lightgbm")
lgbm_tuned = tune_model(lgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4905,1.2963,1.1385,0.5135,0.3154,0.4441
1,0.5047,1.4309,1.1962,0.5501,0.3163,0.4492
2,0.4996,1.5856,1.2592,0.5037,0.3152,0.4440
3,0.5002,1.5345,1.2387,0.5003,0.3236,0.4415
4,0.4990,1.0699,1.0344,0.5507,0.3198,0.4471
5,0.5175,1.2500,1.1181,0.4674,0.3328,0.4584
6,0.5148,2.1219,1.4567,0.4276,0.3212,0.4503
7,0.5081,1.4389,1.1995,0.4957,0.3254,0.4604
8,0.4911,1.0881,1.0431,0.5381,0.3232,0.4507


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4901,1.3044,1.1421,0.5104,0.3147,0.4575
1,0.5116,1.6444,1.2823,0.4830,0.3161,0.4776
2,0.4846,1.5231,1.2342,0.5232,0.3081,0.4653
3,0.4635,1.3026,1.1413,0.5758,0.2940,0.4526
4,0.5111,1.3107,1.1448,0.4495,0.3245,0.4750
5,0.5134,1.3482,1.1611,0.4256,0.3294,0.4634
6,0.5151,2.1978,1.4825,0.4071,0.3220,0.4638
7,0.4995,1.4349,1.1979,0.4970,0.3192,0.4791
8,0.4804,1.1518,1.0732,0.5111,0.3137,0.4662


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [34]:
top4 = compare_models(sort='R2', n_select=4, fold=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.4219,1.3091,1.1431,0.5612,0.2589,0.4549,15.9967
knn,K Neighbors Regressor,0.3704,1.3585,1.1649,0.5448,0.2420,0.5195,54.5667
catboost,CatBoost Regressor,0.4686,1.4130,1.1876,0.5267,0.2915,0.4441,19.7933
lightgbm,Light Gradient Boosting Machine,0.5073,1.5892,1.2603,0.4673,0.3227,0.4593,1.3133
rf,Random Forest Regressor,0.4090,1.6452,1.2824,0.4485,0.2835,0.4591,10.1100
gbr,Gradient Boosting Regressor,0.6829,2.0876,1.4448,0.2999,0.4283,0.4510,5.0033
dt,Decision Tree Regressor,0.4489,2.4854,1.5748,0.1674,0.3396,0.5361,1.0133
et,Extra Trees Regressor,0.6153,2.6536,1.6283,0.1104,0.4309,0.5332,12.5467
lar,Least Angle Regression,0.8801,2.8141,1.6775,0.0564,0.5459,0.5015,0.7233
br,Bayesian Ridge,0.8804,2.8140,1.6774,0.0564,0.5457,0.4993,0.9333


Processing:   0%|          | 0/88 [00:00<?, ?it/s]

In [35]:
stack_model = stack_models(estimator_list=top4, meta_model=lgbm_tuned, fold=5, choose_better=True, optimize='R2')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3494,1.1079,1.0526,0.6210,0.2163,0.4716
1,0.3455,1.1192,1.0579,0.6428,0.2141,0.4681
2,0.3617,0.9187,0.9585,0.6113,0.2312,0.4788
3,0.3601,1.5266,1.2355,0.5346,0.2253,0.4819
4,0.3457,1.2949,1.1379,0.5967,0.2186,0.4746
Mean,0.3525,1.1935,1.0885,0.6013,0.2211,0.4750
Std,0.0070,0.2047,0.0929,0.0365,0.0063,0.0049


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
final_model = finalize_model(stack_model)

In [ ]:
predict = predict_model(final_model, data=test)

In [ ]:
predict

,Month,shopID,itemID,MeanPrice,prediction_label
0,10,0,1000001,363.369720,0.074309
1,10,1,1000001,363.369720,-0.027494
2,10,2,1000001,363.369720,-0.031810
3,10,3,1000001,363.369720,1.972000
4,10,4,1000001,363.369720,2.017525
...,...,...,...,...,...
3055,10,13,3500001,416.063843,-0.005450
3056,10,14,3500001,416.063843,-0.079750
3057,10,15,3500001,416.063843,-0.006014
3058,10,16,3500001,416.063843,-0.088942


In [ ]:
submit = predict["prediction_label"]

In [ ]:
submit

0       0.074309
1      -0.027494
2      -0.031810
3       1.972000
4       2.017525
          ...   
3055   -0.005450
3056   -0.079750
3057   -0.006014
3058   -0.088942
3059    0.023073
Name: prediction_label, Length: 3060, dtype: float32

In [ ]:
submit.to_csv("submit6.csv", index=True, header=False)